In [5]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# Load the preprocessed data
modelling_table = pd.read_csv("preprocessed_data.csv")

# quantile regression
mod = smf.quantreg('total_generation_MWh ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5) + bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=modelling_table)

forecast_models = dict()
for quantile in range(10,100,10):
    forecast_models[f"q{quantile}"] = mod.fit(q=quantile/100,max_iter=2500)
    modelling_table[f"q{quantile}"] = forecast_models[f"q{quantile}"].predict(modelling_table) 
    modelling_table.loc[modelling_table[f"q{quantile}"] < 0, f"q{quantile}"] = 0 #発電量は0以上
    
modelling_table.to_csv("analyzed_data.csv", index=False)

# Save the quantile regression models to files   
for quantile in range(10,100,10):
    forecast_models[f"q{quantile}"].save(f"models/model_q{quantile}.pickle")

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (2500) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
